# Exercise solutions for Chapter 11:
## [Learn Quantum Computing with Python and Q#](https://www.manning.com/books/learn-quantum-computing-with-python-and-q-sharp?a_aid=learn-qc-granade&a_bid=ee23f338)
----
```
// Copyright (c) Sarah Kaiser and Chris Granade.
// Code sample from the book "Learn Quantum Computing with Python and Q#" by
// Sarah Kaiser and Chris Granade, published by Manning Publications Co.
// Book ISBN 9781617296130.
// Code licensed under the MIT License.
```


### Exercise 11.1 

**What are the powers of $11$ when computed$\mod 21$?**

In [59]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Logical;

function Powers(generator : Int, modulus : Int) : Int[] {
    mutable powers = [1];
    mutable acc = generator;
    
    while (acc != 1) {
        set powers += [acc];
        set acc = acc * generator % modulus;
    }
    
    return powers;
}

function PowersOf11Mod21(): Int[] {
    return Powers(11, 21);
}

Powers, PowersOf11Mod21

In [60]:
%simulate PowersOf11Mod21

1, 11, 16, 8, 4, 2

**How long does it take to loop back around to $11^0 = 1$?**

It take $11^6 = 1$ before the powers of $11$ loop back around.

**Does it matter if you take the modulus by $21$ at the end, or whether you compute the modulus at each step?**

No, you can multiply your accumulated value by 11 and then take it mod 21 and then repeat that as many times as needed, and it is the same as modding at the end. The incremental approach scales better as you don't have to store such large numbers and then mod them.

----
### Exercise 11.2

**Using either Python or Q#, try showing that one or both of the potential factor candidates from the last step of Shor's algorithm share factors with the modulus. Use $N = 143$, $g = 19$, and the period $r = 60$.**

So step 6 of Shor's Algorithim is:
> 6. Either $g^{\frac{r}{2}} - 1$ or $g^{\frac{r}{2}} + 1$ shares a factor with $N$.

In [68]:
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;

function SharedFactorsFact(generator : Int, modulus : Int, period : Int) : Unit {
    let halfPower = ExpModI(generator, period / 2, modulus);
    Fact(
        not IsCoprimeI(modulus, halfPower + 1 % modulus) or
        not IsCoprimeI(modulus, halfPower - 1 % modulus)
    , $"Neither {generator}^({period} / 2) + 1 or {generator}^({period} / 2) - 1 shares a factor with {modulus}.");
    
}

function ExerciseFact() : Unit {
    SharedFactorsFact(19, 143, 60);
}

ExerciseFact, SharedFactorsFact

In [69]:
%simulate ExerciseFact

()

----
### Exercise 11.3

**What's the GCD of 35 and 30?**

In [71]:
open Microsoft.Quantum.Math;
function ExerciseGCD() : Int {
    return GreatestCommonDivisorI (35, 30); 
}

ExerciseGCD

In [72]:
%simulate ExerciseGCD

5

This does help you find the factors of $35$ because now you know that $5$ is a factor in common between $35$ and $30$ and thus is also a factor of $35$. You can then divide $35$ by $5$ and get that $7$ is also a factor of $35$.

----
### Exercise 11.4

**What state would your register be in after multiplying by 5 modulo 9?**

> Note: the code below is the same sample code that is in `IntegerFactorization.ipynb`.

In [12]:
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;

operation PrepareSuperpositionOfTwoInts(register : LittleEndian, intPair : (Int, Int)) 
: Unit is Adj + Ctl {
    using(ctrl = Qubit()) {
        H(ctrl);
        
        within{
            X(ctrl);
        } 
        apply{
            Controlled ApplyXorInPlace([ctrl], (Fst(intPair), register));
        }

        Controlled ApplyXorInPlace([ctrl], (Snd(intPair), register));
        (ControlledOnInt(Snd(intPair), Y))(register!, ctrl);  
    }
}


operation MultiplyInSuperpositionMod(
    superpositionInts : (Int, Int), 
    multiplier : Int, 
    modulus : Int
) 
: Int {

    using (target = Qubit[BitSizeI(modulus)]) {

        PrepareSuperpositionOfTwoInts(LittleEndian(target), superpositionInts);
        
        Message("Before multiplication:");
        DumpMachine();
        
        MultiplyByModularInteger(multiplier, modulus, LittleEndian(target));
        
        Message("After multiplication:");
        DumpMachine();
        return MeasureInteger(LittleEndian(target));

    }
}

operation MultiplyInSuperpostionExercise4() :  Int {
    return MultiplyInSuperpositionMod((2, 7), 5, 9);
}

MultiplyInSuperpositionMod, MultiplyInSuperpostionExercise4, PrepareSuperpositionOfTwoInts

In [13]:
 %simulate MultiplyInSuperpostionExercise4

Before multiplication:


|0⟩	0 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0.7071067811865476 + 0𝑖
|3⟩	0 + 0𝑖
|4⟩	0 + 0𝑖
|5⟩	0 + 0𝑖
|6⟩	0 + 0𝑖
|7⟩	0 + -0.7071067811865476𝑖
|8⟩	0 + 0𝑖
|9⟩	0 + 0𝑖
|10⟩	0 + 0𝑖
|11⟩	0 + 0𝑖
|12⟩	0 + 0𝑖
|13⟩	0 + 0𝑖
|14⟩	0 + 0𝑖
|15⟩	0 + 0𝑖

After multiplication:


|0⟩	-1.588064168999351E-32 + -1.3672590894562048E-32𝑖
|1⟩	0.7071067811865562 + -1.0666805631966443E-16𝑖
|2⟩	-8.319612601908143E-33 + 2.7446292116029468E-33𝑖
|3⟩	-2.780574354511211E-32 + -9.458969184507027E-33𝑖
|4⟩	5.719570705496974E-34 + 2.3918631223856762E-32𝑖
|5⟩	-4.6540532372830284E-32 + 9.279892528769384E-32𝑖
|6⟩	-1.8462620191535247E-32 + -4.626440867271962E-34𝑖
|7⟩	1.8169292782583115E-32 + 3.9195257831161963E-32𝑖
|8⟩	-1.450825192588669E-16 + -0.7071067811865577𝑖
|9⟩	-2.617496027443308E-32 + -5.902359212409753E-33𝑖
|10⟩	2.1610701549034368E-17 + 1.4369815823408824E-17𝑖
|11⟩	-5.817150284069754E-33 + -1.0167814855897907E-32𝑖
|12⟩	-1.2269619807071403E-32 + 2.1909279410141568E-33𝑖
|13⟩	-1.5424392615683752E-32 + 4.1694076824636494E-33𝑖
|14⟩	-4.314819704341814E-33 + -9.646808941518633E-33𝑖
|15⟩	4.318711737853307E-33 + 5.7808809161550995E-33𝑖

1

You can see that after multiplication, the state is $\frac{1}{\sqrt{2}}\left(\left|1\right\rangle + \left|8\right\rangle\right)$

----
### Exercise 11.5

**Try to multiply by 3 modulo 9, you'll get an error; why?**

In [10]:
operation MultiplyInSuperpostionExercise5() :  Int {
    // Here we are multiplying 3 by a superposition of 2 and 
    // 3 all mod 8.
    return MultiplyInSuperpositionMod((2, 7), 3, 9);
}

MultiplyInSuperpostionExercise5

In [11]:
%simulate MultiplyInSuperpostionExercise5

Before multiplication:


|0⟩	0 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0.7071067811865476 + 0𝑖
|3⟩	0 + 0𝑖
|4⟩	0 + 0𝑖
|5⟩	0 + 0𝑖
|6⟩	0 + 0𝑖
|7⟩	0 + -0.7071067811865476𝑖
|8⟩	0 + 0𝑖
|9⟩	0 + 0𝑖
|10⟩	0 + 0𝑖
|11⟩	0 + 0𝑖
|12⟩	0 + 0𝑖
|13⟩	0 + 0𝑖
|14⟩	0 + 0𝑖
|15⟩	0 + 0𝑖

Source,Callable
D:\a\1\s\submodules\QuantumLibraries\Standard\src\Arithmetic\Modular.qs:236,Microsoft.Quantum.Arithmetic.MultiplyByModularInteger
C:\snippet_16a1ca42-2a78-47e7-a8b4-785625f8f48c.qs:0,MultiplyInSuperpositionMod
(notebook),MultiplyInSuperpostionExercise5


False ≠ True: `constMultiplier` and `modulus` must be co-prime


<details> Multiplying by three mod nine is not reversible.
For instance, both one times three and four times three mod nine give zero, even though one and four aren't equal mod nine.
Since classical functions have to be reversible in order to be represented by quantum operations, the `MultiplyByModularInteger` raises an error in this case. <summary> Expand this for the decoded anwser </summary></details>
   

----
### Appendix

_The following cell logs what version of the components this was last tested with._

In [1]:
%version

Component,Version
iqsharp,0.11.2003.3107
Jupyter Core,1.2.36563.0
.NET Runtime,".NETCoreApp,Version=v3.1"
